# Assignment

The goal of this assignment is to understand and implement different types of regression and classification algorithms using Python. The specific algorithms to be used are:

### Algorithms: 
* Regression 
* Logistic Regression 
* Naive Bayes

### Tasks: 
1. Perform EDA on the dataset and pre-processing if necessary.
2. Train a linear regression , logistic regression and naive bayes model using the dataset.
3. Evaluate the model's performance using appropriate metrics (e.g. accuracy, precision, recall, etc.).
4. Use the trained model to make predictions on unseen data.
5. Compare the perfomance of Linear Regression, Logistic Regression and Naive Bayes.
6. Analyze the results and draw conclusions.


### DATASET - *AI4I 2020 Predictive Maintenance*

#### Description

The AI4I 2020 Predictive Maintenance Dataset is a synthetic dataset that reflects real predictive maintenance data encountered in industry. Since real predictive maintenance datasets are generally difficult to obtain and in particular difficult to publish, we present and provide a synthetic dataset that reflects real predictive maintenance encountered in industry to the best of our knowledge.

#### Attribute Information:

The dataset consists of 10000 data points stored as rows with 14 features in columns

- UID: unique identifier ranging from 1 to 10000
- product ID: consisting of a letter L, M, or H for low (50% of all products), medium (30%) and high (20%) as product quality variants and a variant-specific serial number
- air temperature [K]: generated using a random walk process later normalized to a standard deviation of 2 K around 300 K
- process temperature [K]: generated using a random walk process normalized to a standard deviation of 1 K, added to the air temperature plus 10 K.
- rotational speed [rpm]: calculated from a power of 2860 W, overlaid with a normally distributed noise
- torque [Nm]: torque values are normally distributed around 40 Nm with a f = 10 Nm and no negative values.
- tool wear [min]: The quality variants H/M/L add 5/3/2 minutes of tool wear to the used tool in the process.
- 'machine failure' label that indicates, whether the machine has failed in this particular datapoint for any of the following failure modes are true.

The machine failure consists of five independent failure modes :

- tool wear failure (TWF): the tool will be replaced of fail at a randomly selected tool wear time between 200-240 mins (120 times in our dataset). At this point in time, the tool is replaced 69 times, and fails 51 times (randomly assigned).

- heat dissipation failure (HDF): heat dissipation causes a process failure, if the difference between air- and process temperature is below 8.6 K and the tools rotational speed is below 1380 rpm. This is the case for 115 data points.
- power failure (PWF): the product of torque and rotational speed (in rad/s) equals the power required for the process. If this power is below 3500 W or above 9000 W, the process fails, which is the case 95 times in our dataset.
- overstrain failure (OSF): if the product of tool wear and torque exceeds 11,000 minNm for the L product variant (12,000 M, 13,000 H), the process fails due to overstrain. This is true for 98 datapoints.
- random failures (RNF): each process has a chance of 0,1 % to fail regardless of its process parameters. This is the case for only 5 datapoints, less than could be expected for 10,000 datapoints in our dataset.

If at least one of the above failure modes is true, the process fails and the 'machine failure' label is set to 1. It is therefore not transparent to the machine learning method, which of the failure modes has caused the process to fail


# Inputs 
- UID
- Product ID
- Air temperature [K]
- Process temperature [K]
- Rotational speed [rpm]
- Torque [Nm]
- Tool wear [min]

# Output
- Machine failure (1 = failed, 0 = not failed)


Dataset Link: https://archive.ics.uci.edu/ml/datasets/AI4I+2020+Predictive+Maintenance+Dataset

### Submission

Kindly play around with the exploratory data analysis and model building. You can also try to improve the model's performance by tuning the hyperparameters.

Submit your assignment as a Jupyter notebook on your GitHub repository. The notebook should contain all the code and outputs. You can also submit a PDF version of the notebook. 

 





In [ ]:
# Load the data from the file 'ai4i2020.csv' into a Pandas DataFrame

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [ ]:
df = pd.read_csv('ai4i2020.csv')
df.head()

In [ ]:
# Preprocess the data

df.describe()


In [ ]:
df.info()

## Exploratory Data Analysis

- The dataset has 10000 rows and 14 columns and we perform EDA on the dataset to understand the data better by checking for missing values, outliers, and other anomalies in the data.There is no missing value or duplicate value in the dataset.

- The dataset has 7 numerical columns and 7 categorical columns. The numerical columns are 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]', 'Machine failure', 'UID'. The categorical columns are 'Product ID', 'Machine failure', 'Tool wear failure', 'Heat dissipation failure', 'Power failure', 'Overstrain failure', 'Random failure'.

- We check for outliers in the numerical columns using boxplots. We find that there are outliers in the 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]' columns. We remove the outliers using the IQR method.

- Convert the categorical columns to numerical columns using label encoding.

In [ ]:
# Check for duplicate values and drop them
print("Checking for duplicate values: ", df.duplicated().sum() != 0)


In [ ]:
# set the numeric columns data type to float

df['Process temperature [K]'] = df['Process temperature [K]'].astype(float)
df['Rotational speed [rpm]'] = df['Rotational speed [rpm]'].astype(float)
df['Torque [Nm]'] = df['Torque [Nm]'].astype(float)
df['Tool wear [min]'] = df['Tool wear [min]'].astype(float)
df['Machine failure'] = df['Machine failure'].astype(float)


ds = df["Type"].value_counts().reset_index()[:28]
ds





In [ ]:
# Remove first character and set to numeric dtype
df['Product ID'] = df['Product ID'].apply(lambda x: x[1:])
df['Product ID'] = pd.to_numeric(df['Product ID'])


# Convert the Type column M,L,H to 0,1,2
df['Type'] = df['Type'].apply(lambda x: 0 if x == 'L' else 1 if x == 'M' else 2)



# Show both plots in the same figure using subplots
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
ax[0].pie(df['Type'].value_counts(), labels=df['Type'].value_counts().index, autopct='%1.1f%%')
ax[0].set_title('Pie chart of Type')
sns.histplot(data=df, x='Product ID', hue='Type', ax=ax[1])
ax[1].set_title('Histogram of Product ID')
plt.show()





In [ ]:

# Drop the UDI and Product ID column as it is not needed for the model training
df_ = df.copy()
df = df_.drop(['UDI', 'Product ID'], axis=1)

# Check for missing values
df.isnull().sum()

# Print count of Unique values of each column
for col in df.columns:
    print(col, df[col].nunique())

In [ ]:
# Plot the correlation matrix 

corr = df.corr()
plt.figure(figsize=(10, 10))
sns.heatmap(corr, annot=True, cmap='coolwarm', linewidths=0.5, fmt='.2f')
plt.show()


In [ ]:
# Plot visualizations of the data and their correlations with the target variable

numeric_cols = ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]','Type']
sns.pairplot(df.loc[:,numeric_cols],hue="Type",diag_kind='kde',kind='scatter')
plt.show()
    


# Feature Engineering

- We now analyse the correlation between the features and the target variable 'Machine failure'. We find that the 'Tool wear [min]' column has the highest correlation with the target variable. We also find that the 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]' columns have a high correlation with the target variable.

    

In [ ]:
# Standardize the data 
from sklearn.preprocessing import MinMaxScaler
X = df.drop('Type', axis=1)
y = df['Type']

print(X.shape, y.shape)



# Split the data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=21)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
    

 # Print the shapes of the training and testing sets and top 5 rows of the training set , X_train, y_train and X_test, y_test
print("----Training Data ----")
print(X_train.shape, y_train.shape)
print("----Test Data ----")
print(X_test.shape, y_test.shape)

for i in range(5):
    print("Training Set:", X_train[i], y_train.iloc[i])
    print("Testing Set:", X_test[i], y_test.iloc[i])
    




## Model Building

### Linear Regression

- Linear regression is a linear approach to modelling the relationship between a scalar response and one or more explanatory variables. The case of one explanatory variable is called simple linear regression. For more than one explanatory variable, the process is called multiple linear regression.

- The mathematical equation for linear regression is:

$$y = \beta_0 + \beta_1x$$

- The goal of linear regression is to find the best fit line for the data. The best fit line is the line that has the least error. The error is the difference between the actual value and the predicted value. The error is calculated using the mean squared error function.

- The mean squared error function is:

$$MSE = \frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{y_i})^2$$  


### Logistic Regression 

- Logistic regression is a statistical model that in its basic form uses a logistic function to model a binary dependent variable, although many more complex extensions exist. In regression analysis, logistic regression (or logit regression) is estimating the parameters of a logistic model (a form of binary regression).

- The mathematical equation for logistic regression is:

$$\frac{1}{1+e^{-z}}$$

- The goal of logistic regression is to find the best fit line for the data. The best fit line is the line that has the least error. The error is the difference between the actual value and the predicted value. The error is calculated using the mean squared error function.

- The mean squared error function is:

$$MSE = \frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{y_i})^2$$


### Naive Bayes

- Naive Bayes is a classification technique based on Bayes’ Theorem with an assumption of independence among predictors. In simple terms, a Naive Bayes classifier assumes that the presence of a particular feature in a class is unrelated to the presence of any other feature.


- The mathematical equation for naive bayes is:

$$P(A|B) = \frac{P(B|A)P(A)}{P(B)}$$

- The goal of naive bayes is to find the best fit line for the data. The best fit line is the line that has the least error. The error is the difference between the actual value and the predicted value. The error is calculated using the mean squared error function.

- The mean squared error function is:

$$MSE = \frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{y_i})^2$$








In [ ]:
# Train a linear regression model

from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

# TODO: Train a linear regression model and print the model's coefficients and intercept use lr as the model name 

# lr : Linear Regression model  , y_pred : Predicted values of y_test  


# Evaluation metrics
lr_mse = mean_squared_error(y_test, y_pred)
lr_r2 = r2_score(y_test, y_pred)

# Store the results in a dictionary
results = {'Linear Regression': [lr_mse, lr_r2]}
results



In [ ]:
# Train a Logistic Regression model

# TODO: Train a logistic regression model 

# lr : Logistic Regression model  , y_pred_lr : Predicted values of y_test 


# Evaluation metrics

y_pred_lr = lr.predict(X_test)
log_train = round(lr.score(X_train, y_train) * 100, 2)
log_accuracy = round(accuracy_score(y_pred_lr, y_test) * 100, 2)


print("Training Accuracy    :",log_train ,"%")
print("Model Accuracy Score :",log_accuracy ,"%")
print("\033[1m--------------------------------------------------------\033[0m")
print("Classification_Report: \n",classification_report(y_test,y_pred_lr,zero_division=0))
print("\033[1m--------------------------------------------------------\033[0m")
plot_confusion_matrix(lr, X_test, y_test)
plt.title('Confusion Matrix')

In [ ]:
# Implement a Naive Bayes Classifier

"TODO: Train a Naive Bayes model and print the model's coefficients and intercept use lr as the model name "

" gnb : Naive Bayes Model  , y_pred_gnb : Predicted values of y_test "


gnb_train = round(gnb.score(X_train, y_train) * 100, 2)

gnb_accuracy = round(accuracy_score(y_pred_gnb, y_test) * 100, 2)

print("Training Accuracy    :",gnb_train ,"%")
print("Model Accuracy Score :",gnb_accuracy ,"%")

print("\033[1m--------------------------------------------------------\033[0m")

print("Classification_Report: \n",classification_report(y_test,y_pred_gnb))

print("\033[1m--------------------------------------------------------\033[0m")

plot_confusion_matrix(gnb, X_test, y_test)

plt.title('Confusion Matrix')

cm = confusion_matrix(y_test, y_pred_gnb)





In [ ]:
# Compare the results of the models

results['Logistic Regression'] = [log_accuracy, log_train]
results['Naive Bayes'] = [gnb_accuracy, gnb_train]
results_df = pd.DataFrame(results, index=['Accuracy', 'Training Accuracy']).T
results_df
